## Deribit API Request  - For BTC and ETH Options Data
#### ALL Options data 

### Specfic Settlement Period ( Day, Week, Month) 

In [2]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
from tqdm import tqdm
import concurrent.futures

# Functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1. list of traded options for the selected coin;
                        2. list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # Get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # Get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period

def fetch_option_data(option_name):
    """Fetch the option data for a given option name."""
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    return pd.json_normalize(result['result'])

def extract_details(instrument_name, coin):
    """
    Extract expiration date and strike price from instrument name.
    Adjusts to include selected coin (e.g., 'BTC' or 'ETH').
    """
    match = re.match(fr"{coin}-(\d+[A-Z]{{3}}\d+)-(\d+)-[CP]", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        return expiration_date, strike_price
    return None, None

def get_option_data(coin, settlement_per):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas DataFrame with all option data for a given coin, filtered for options with specified settlement.
    """
    # Get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # Filter options that have the specified settlement period
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == settlement_per]

    # Initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = settlement_per
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # Finalize DataFrame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # Remove unnecessary columns
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    coin_df['Expiration Date'], coin_df['Strike Price'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))

    today = datetime.today()
    coin_df['Time to Expiration'] = coin_df['Expiration Date'].apply(lambda x: (datetime.strptime(x, '%d%b%y') - today).days / 365 if x else None)

    coin_df.to_csv('data/data.csv', index=False)
 
    pbar.close()

    return coin_df


In [5]:
data = get_option_data('ETH','month')

 99%|█████████▉| 579/584 [01:15<00:00, 12.50it/s]C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_17532\3260114596.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 584/584 [01:15<00:00,  7.75it/s]


In [4]:
data

,best_bid_amount,best_ask_amount,ask_iv,bid_iv,underlying_index,underlying_price,mark_iv,best_bid_price,best_ask_price,interest_rate,...,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period,Expiration Date,Strike Price,Time to Expiration
0,454.0,507.0,119.13,102.04,SYN.ETH-30OCT24,2636.2189,111.34,0.0001,0.0003,0.0,...,-0.01551,216.76,323.0,-75.0000,0.0001,0.0004,day,30OCT24,2450,0.00000
0,53.0,559.0,115.79,0.00,SYN.ETH-30OCT24,2636.2189,64.25,0.0325,0.0360,0.0,...,0.95494,81.24,1.0,0.0000,0.0310,0.0310,day,30OCT24,2550,0.00000
0,0.0,718.0,133.18,0.00,SYN.ETH-30OCT24,2636.2189,128.00,0.0000,0.0003,0.0,...,-0.01612,1.84,7.0,0.0000,0.0001,0.0001,day,30OCT24,2425,0.00000
0,711.0,561.0,158.51,0.00,SYN.ETH-30OCT24,2636.2189,111.34,0.0705,0.0720,0.0,...,0.98457,0.00,0.0,NaN,NaN,NaN,day,30OCT24,2450,0.00000
0,31.0,553.0,71.15,57.45,SYN.ETH-30OCT24,2636.2189,64.25,0.0002,0.0006,0.0,...,-0.04506,61743.27,3702.0,-79.0323,0.0013,0.0071,day,30OCT24,2550,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,603.0,639.0,76.24,0.00,SYN.ETH-31OCT24,2640.1930,58.57,0.0420,0.0465,0.0,...,-0.87053,0.00,0.0,NaN,NaN,NaN,day,31OCT24,2750,0.00274
0,0.0,575.0,85.77,0.00,SYN.ETH-31OCT24,2640.1930,62.64,0.0000,0.0640,0.0,...,-0.93660,0.00,0.0,NaN,NaN,NaN,day,31OCT24,2800,0.00274
0,20.0,20.0,65.94,59.32,SYN.ETH-31OCT24,2640.1930,62.64,0.0008,0.0013,0.0,...,0.06350,0.00,0.0,NaN,NaN,NaN,day,31OCT24,2800,0.00274
0,0.0,0.0,0.00,0.00,SYN.ETH-31OCT24,2640.1930,67.49,0.0000,0.0000,0.0,...,-0.95330,0.00,0.0,NaN,NaN,NaN,day,31OCT24,2850,0.00274
